## 3.2 Métodos diretos: decomposição LU

Os métodos diretos para resolução de sistemas lineares teoricamente dão a solução exata do sistema em um número finito de passos. Na prática, as soluções podem sofrer erros de arredondamento devido à aritmética que está sendo usada. 

Os métodos diretos de eliminação consistem no processo 
de transformar um sistema de equações lineares $Ax=b$, em que $A= [a_{ij}]_i,j=1,..,n$, 
$x=[x_1,x_2,...,x_n]^t$ e $b=[b_1, b_2,...,b_n]^t$, em um sistema equivalente, 
aplicando operações elementares sobre as linhas da matriz aumentada $[A \mid b]$. O sistema equivalente obtido ao final do processo, deve ser de 
resolução mais simples ou imediata, por substituição direta. 

O método de eliminação de Gauss é o método direto usado para escalonar a matriz que representa um sistema linear com o objetivo de obter um ou mais sistemas triangulares, os quais podem ser resolvidos por subtituições subsequentes. Outros métodos diretos podem envolver fatorar a matriz dos coeficientes do sistema em um produto de duas ou mais matrizes, como é o caso do método por fatoração LU, que será abordado nesse texto. 


Antes disso, vamos relembrar as função já desenvolvidas nas aulas anteriores.
#### Funções para escalonar uma matriz e para resolver sistemas triangulares

In [1]:
import numpy as np

# função para escalonar (sem pivoteamento)
def escalona(M):
    n = len(M)
    for j in range(n-1):
        for i in range(j,n-1):            
            m = M[i+1,j]/M[j,j]
            M[i+1] = M[i+1]-m*M[j]
    return (M)

# função para sistema triangular inferior
def solve_tril(L,b):
    y = np.empty(len(b))
    for i in range(len(b)): 
        y[i] = (b[i] - np.dot(L[i,0:i],y[0:i]))/L[i][i] 
    return y

# função para sistema triangular superior
def solve_triu(U,y):
    n=len(y)
    x = np.empty(n)
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - np.sum(U[i,i+1:n+1]*x[i+1:n+1]))/U[i,i]
    return x

**Exemplo 1:** Vamos neste exemplo revisar o sistema linear abaixo, já estudado anteriormente ([fonte](https://integrada.minhabiblioteca.com.br/reader/books/9788522123414/pageid/418)). 
$$
\begin{aligned}
x_1-x_2+2 x_3-x_4 & =-8, \\
2 x_1-2 x_2+3 x_3-3 x_4 & =-20, \\
x_1+x_2+x_3 & =-2, \\
x_1-x_2+4 x_3+3 x_4 & =4 .
\end{aligned}
$$


Para isso, vamos analisar o escalonamento, coluna por coluna.

In [2]:
# escrevendo a matriz ampliada
M = np.array([[1,-1,2,-1,-8],
              [2,-2,3,-3,-20],
              [1,1,1,0,-2],
              [1,-1,4,3,4]], dtype=float)

In [3]:
# zerando os elementos da primeira coluna 
for i in range(3):            
    m = M[i+1,0]/M[0,0]
    M[i+1] = M[i+1] - m*M[0]
print (M)

[[ 1. -1.  2. -1. -8.]
 [ 0.  0. -1. -1. -4.]
 [ 0.  2. -1.  1.  6.]
 [ 0.  0.  2.  4. 12.]]


Observemos que o elemento pivô na segunda linha é zero, portanto, para que o processo de eliminação possa ser continuado, é necessário realizar uma troca de linhas.

In [4]:
M[[1,2]] = M[[2,1]]
print(M)

[[ 1. -1.  2. -1. -8.]
 [ 0.  2. -1.  1.  6.]
 [ 0.  0. -1. -1. -4.]
 [ 0.  0.  2.  4. 12.]]


Agora podemos seguir o escalonamento na terceira coluna.

In [5]:
# zerando os elementos da terceira coluna 
m = M[3,2]/M[2,2]
M[3] = M[3] - m*M[2]
print (M)

[[ 1. -1.  2. -1. -8.]
 [ 0.  2. -1.  1.  6.]
 [ 0.  0. -1. -1. -4.]
 [ 0.  0.  0.  2.  4.]]


In [6]:
# solução do sistema
x = solve_triu(M[:,:-1], M[:,-1])
print(x)

[-7.  3.  2.  2.]


***

Uma função bastante útil para tratar esse tipo de situação é a função `numpy.roll()`, que "rola" os elementos do array ao longo do eixo especificado. Se o elemento a ser desclocado estiver na última posição, ele será levado de volta para a primeira posição. Veja um exemplo a seguir.

In [7]:
# rolando as linhas de M para baixo uma vez
np.roll(M, 1, axis=0)

array([[ 0.,  0.,  0.,  2.,  4.],
       [ 1., -1.,  2., -1., -8.],
       [ 0.,  2., -1.,  1.,  6.],
       [ 0.,  0., -1., -1., -4.]])

In [8]:
# rolando as linhas de M para cima duas vezes
np.roll(M, -2, axis=0)

array([[ 0.,  0., -1., -1., -4.],
       [ 0.,  0.,  0.,  2.,  4.],
       [ 1., -1.,  2., -1., -8.],
       [ 0.,  2., -1.,  1.,  6.]])

**Exercício 1:** 
Use a função `roll` para modificar a função `escalona` mostrada abaixo, de modo que elementos nulos na posição dos pivôs sejam substituídos pela troca de linhas da matriz.
Use a matriz do Exemplo 1 para testar.


In [9]:
M = np.array([[1,-1,2,-1,-8],
              [2,-2,3,-3,-20],
              [1,1,1,0,-2],
              [1,-1,4,3,4]], dtype=float)

In [10]:
def escalona2(M):
    n = len(M)
    for j in range(n-1):
        for i in range(j,n-1): 
            # rolando as linhas
            while M[j,j]==0:
                M[j:,:] = np.roll(M[j:,:], -1, axis=0)
            m = M[i+1,j]/M[j,j]
            M[i+1] = M[i+1]-m*M[j]
    return (M)

In [11]:
M = escalona2(M)
print(M)

[[ 1. -1.  2. -1. -8.]
 [ 0.  2. -1.  1.  6.]
 [ 0.  0.  2.  4. 12.]
 [ 0.  0.  0.  1.  2.]]


In [12]:
# solução do sistema
x = solve_triu(M[:,:-1], M[:,-1])
print(x)

[-7.  3.  2.  2.]


Observe que a matriz escalonada é diferente da matriz obtida no Exemplo 1 pois a troca de linha ocorreu de forma diferente durante o processo. Mesmo assim, o resultado final é o mesmo, como já era esperado.

**Exercício 2:**
Resolva o seguinte sistema pelo método de eliminação de Gauss.

$$
\begin{aligned}
2 x_1+x_2-x_3+x_4-3 x_5 & =7 \\
x_1+2 x_3-x_4+x_5 & =2 \\
-2 x_2-x_3+x_4-x_5 & =-5 \\
3 x_1+x_2-4 x_3+5 x_5 & =6 \\
x_1-x_2-x_3-x_4+x_5 & =3
\end{aligned}
$$

In [13]:
M = np.array([[2,1,-1,1,-3,7],
             [1,0,2,-1,1,2],
             [0,-2,-1,1,-1,-5],
             [3,1,-4,0,5,6],
             [1,-1,-1,-1,1,3]], dtype=float)

In [14]:
Mt = escalona(M.copy())
np.round(Mt,3)

array([[  2.   ,   1.   ,  -1.   ,   1.   ,  -3.   ,   7.   ],
       [  0.   ,  -0.5  ,   2.5  ,  -1.5  ,   2.5  ,  -1.5  ],
       [  0.   ,   0.   , -11.   ,   7.   , -11.   ,   1.   ],
       [  0.   ,   0.   ,   0.   ,  -3.182,  12.   ,  -3.455],
       [  0.   ,   0.   ,   0.   ,   0.   ,  -4.886,   5.543]])

In [15]:
# solução do sistema
x = solve_triu(Mt[:,:-1], Mt[:,-1])
print(x)

[ 1.91812865  1.96491228 -0.98830409 -3.19298246 -1.13450292]


In [16]:
M

array([[ 2.,  1., -1.,  1., -3.,  7.],
       [ 1.,  0.,  2., -1.,  1.,  2.],
       [ 0., -2., -1.,  1., -1., -5.],
       [ 3.,  1., -4.,  0.,  5.,  6.],
       [ 1., -1., -1., -1.,  1.,  3.]])

In [17]:
M[:,:-1]@x

array([ 7.,  2., -5.,  6.,  3.])

**Exercício 3:** Use o algoritmo de eliminação de Gauss para resolver os seguintes sistemas lineares, se possível, e determine se são necessárias trocas de linhas ([fonte](https://integrada.minhabiblioteca.com.br/reader/books/9788522123414/pageid/424))

a)$
\begin{aligned}
x_1+x_2+x_4 & =2, \\
2 x_1+x_2-x_3+x_4 & =1, \\
4 x_1-x_2-2 x_3+2 x_4 & =0, \\
3 x_1-x_2-x_3+2 x_4 & =-3 .
\end{aligned}
\quad \quad$
b)
$
\begin{aligned}
x_1-\frac{1}{2} x_2+x_3 & =4 \\
2 x_1-x_2-x_3+x_4 & =5 \\
x_1+x_2+\frac{1}{2} x_3 & =2 \\
x_1-\frac{1}{2} x_2+x_3+x_4 & =5
\end{aligned}
\quad \quad$
c)
$\begin{aligned} 
x_1+x_2+x_4 & =2, \\ 
2 x_1+x_2-x_3+x_4 & =1, \\ 
-x_1+2 x_2+3 x_3-x_4 & =4, \\ 
3 x_1-x_2-x_3+2 x_4 & =-3 .
\end{aligned}$

In [18]:
M1 = np.array([[1,1,0,1,2],[2,1,-1,1,1],[4,-1,-2,2,0],[3,-1,-1,2,-3]],dtype=float)
M2 = np.array([[1,-0.5,1,0,4],[2,-1,-1,1,5],[1,1,0.5,0,2],[1,-0.5,1,1,5]],dtype=float)
M3 = np.array([[1,1,0,1,2],[2,1,-1,1,1],[-1,2,3,-1,4],[3,-1,-1,2,-3]],dtype=float)

*Solução letra a):*

In [19]:
M1e = escalona(M1)
M1e

array([[ 1.,  1.,  0.,  1.,  2.],
       [ 0., -1., -1., -1., -3.],
       [ 0.,  0.,  3.,  3.,  7.],
       [ 0.,  0.,  0.,  0., -4.]])

Sistema impossível.

*Solução letra b):*

In [20]:
M2e = escalona(M2.copy())
M2e

<ipython-input-1-f865995d359a>:8: RuntimeWarning: divide by zero encountered in double_scalars
  m = M[i+1,j]/M[j,j]
<ipython-input-1-f865995d359a>:9: RuntimeWarning: invalid value encountered in multiply
  M[i+1] = M[i+1]-m*M[j]
<ipython-input-1-f865995d359a>:8: RuntimeWarning: invalid value encountered in double_scalars
  m = M[i+1,j]/M[j,j]


array([[ 1. , -0.5,  1. ,  0. ,  4. ],
       [ 0. ,  0. , -3. ,  1. , -3. ],
       [ nan,  nan,  inf, -inf,  inf],
       [ nan,  nan,  nan,  nan,  nan]])

In [21]:
M2e = escalona2(M2.copy())
M2e

array([[ 1. , -0.5,  1. ,  0. ,  4. ],
       [ 0. ,  1.5, -0.5,  0. , -2. ],
       [ 0. ,  0. , -3. ,  1. , -3. ],
       [ 0. ,  0. ,  0. ,  1. ,  1. ]])

In [22]:
# solução do sistema
x = solve_triu(M2e[:,:-1], M2e[:,-1])
print(x)

[ 2.22222222 -0.88888889  1.33333333  1.        ]


In [23]:
M2[:,:-1]@x

array([4., 5., 2., 5.])

*Solução letra c):*

In [24]:
M3

array([[ 1.,  1.,  0.,  1.,  2.],
       [ 2.,  1., -1.,  1.,  1.],
       [-1.,  2.,  3., -1.,  4.],
       [ 3., -1., -1.,  2., -3.]])

In [25]:
M3e = escalona2(M3.copy())
M3e

array([[ 1.,  1.,  0.,  1.,  2.],
       [ 0., -1., -1., -1., -3.],
       [ 0.,  0.,  3.,  3.,  3.],
       [ 0.,  0.,  0., -3., -3.]])

In [26]:
# solução do sistema
x = solve_triu(M3e[:,:-1], M3e[:,-1])
print(x)

[-1.  2.  0.  1.]


In [27]:
M3[:,:-1]@x

array([ 2.,  1.,  4., -3.])

***

### DecomposiçãoLU

Os passos utilizados para resolver um sistema da forma $Ax = b$ por eliminação de Gauss podem ser utilizados para fatorar uma matriz $A$ em um produto de duas matrizes triangulares $L$ (triangular inferior) e $U$ (triangular superior).  

**Exemplo 3:** Vejamos como resolver o sistema 

$$
\left[\begin{array}{lll}
2 & 0 & 1 \\
0 & 2 & 1 \\
1 & 1 & 3
\end{array}\right]\left[\begin{array}{l}
x_1 \\
x_2 \\
x_3
\end{array}\right]=\left[\begin{array}{l}
3 \\
3 \\
5
\end{array}\right]
$$

usando o processo de fatoração $LU$.

In [28]:
# criamos as matrizes
A = np.array([[2,0,1],[0,2,1],[1,1,3]], dtype=float)
b = np.array([3,3,5], dtype=float)

Então, vamos elimintar os elementos da 1ª coluna, abaixo do pivô:

In [29]:
# segunda linha
m21 = A[1,0]/A[0,0]
A[1] = A[1] - m21*A[0]

#terceira linha
m31 = A[2,0]/A[0,0]
A[2] = A[2] - m31*A[0]

print (A)

[[2.  0.  1. ]
 [0.  2.  1. ]
 [0.  1.  2.5]]


Observe que o resultado é o mesmo que se multiplicar a matriz

$$M=
\left[\begin{array}{lll}
1 & 0 & 0 \\
-m21 & 1 & 0 \\
-m31 & 0 & 1
\end{array}\right]
=
\left[\begin{array}{lll}
1 & 0 & 0 \\
0 & 1 & 0 \\
-0.5 & 0 & 1
\end{array}\right]
$$.

pela matriz $A$ como é mostrado abaixo

In [30]:
M = np.array([[1,0,0],[0,1,0],[-0.5,0,1]])
A =np.array([[2,0,1],[0,2,1],[1,1,3]])
A = M@A
print(A)

[[2.  0.  1. ]
 [0.  2.  1. ]
 [0.  1.  2.5]]


Por fim, concluímos a eliminação de Gauss fazendo:

In [31]:
#terceira linha segunda coluna
m32 = A[2,1]/A[1,1]
A[2] = A[2] - m32*A[1]
print (A)

[[2. 0. 1.]
 [0. 2. 1.]
 [0. 0. 2.]]


Podemos chegar a esse mesmo resultado fazendo  

$$
\left[\begin{array}{lll}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & -m32 & 1
\end{array}\right]
\left[\begin{array}{lll}
1 & 0 & 0 \\
-m21 & 1 & 0 \\
-m31 & 0 & 1
\end{array}\right]
\left[\begin{array}{lll}
2 & 0 & 1 \\
0 & 2 & 1 \\
1 & 1 & 3
\end{array}\right]
=
\left[\begin{array}{lll}
1 & 0 & 0 \\
-m21 & 1 & 0 \\
-m31 & -m32 & 1
\end{array}\right]
\left[\begin{array}{lll}
2 & 0 & 1 \\
0 & 2 & 1 \\
1 & 1 & 3
\end{array}\right]
=
\left[\begin{array}{lll}
2 & 0 & 1 \\
0 & 2 & 1 \\
0 & 0 & 2
\end{array}\right]
$$


Além disso, note que 

$$
\left[\begin{array}{lll}
1 & 0 & 0 \\
-m21 & 1 & 0 \\
-m31 & -m32 & 1
\end{array}\right]^{-1}=
\left[\begin{array}{lll}
1 & 0 & 0 \\
-m21 & 1 & 0 \\
-m31 & -m32 & 1
\end{array}\right]$$

e portanto podemos escrever a fatoração como 

$$
A=LU \quad \rightarrow \quad 
\left[\begin{array}{lll}
2 & 0 & 1 \\
0 & 2 & 1 \\
1 & 1 & 3
\end{array}\right]=
\left[\begin{array}{lll}
1 & 0 & 0 \\
0 & 1 & 0 \\
-0.5 & -0.5 & 1
\end{array}\right]
\left[\begin{array}{lll}
2 & 0 & 1 \\
0 & 2 & 1 \\
0 & 0 & 2
\end{array}\right]
$$

Como pode ser veificado como é mostrado abaixo

In [32]:
L = np.array([[1,0,0],[0,1,0],[0.5,0.5,1]])
U = np.array([[2,0,1],[0,2,1],[0,0,2]])
L@U

array([[2., 0., 1.],
       [0., 2., 1.],
       [1., 1., 3.]])

Para obter a solução do sistema linear $Ax=b$ por decomposição $LU$ fazemos

$$Ax=b \rightarrow LUx=b$$

escrevendo $Ux=y$ tem-se dois sitemas triangulares 

<p>
<div align="center">
$Ly=b\,\,\,$ e, então, $\,\,\,Ux=y$
</div>

Resolvendo o sistema $Ly=b$ obtem-se $y$, então, resolvendo o sistema $Ux=y$, obtem-se a solução procurada $x = (x_1,...,x_n)^T$.

Assim, a resolução do sistema $Ax=b$ é feita resolvendo-se o sistema triangular inferior e, em seguida, o sistema triagular superior.

In [33]:
y = solve_tril(L,b)
print(y)

[3. 3. 2.]


In [34]:
x = solve_triu(U,y)
print(x)

[1. 1. 1.]


Agora, podemos sintetizar os resultados escrevendo uma funação para fatorar uma matriz por eliminação de Gauss , fazendo:

In [35]:
def fatoraLU(A):
    L = np.eye(len(A))
    U = A.copy()
    n = len(A)
    for j in range(n-1):
        for i in range(j,n-1):            
            m = U[i+1,j]/U[j,j]
            U[i+1] = U[i+1]-m*U[j]
            L[i+1,j] = m 
    return (L,U)

In [36]:
# testando
A = np.array([[2,0,1],[0,2,1],[1,1,3]], dtype=float)
L,U = fatoraLU(A) 
print(L)
print(U)

[[1.  0.  0. ]
 [0.  1.  0. ]
 [0.5 0.5 1. ]]
[[2. 0. 1.]
 [0. 2. 1.]
 [0. 0. 2.]]


**Exemplo 3:** Agora vamos considerar novamente o sistema do exemplo 1 e resolver usando fatoração LU.

In [37]:
M = np.array([[2,1,-1,1,-3,7],
             [1,0,2,-1,1,2],
             [0,-2,-1,1,-1,-5],
             [3,1,-4,0,5,6],
             [1,-1,-1,-1,1,3]], dtype=float)

In [38]:
A = M[:,0:5]
b = M[:,-1]

In [39]:
L,U = fatoraLU(A)
y = solve_tril(L,b)
x = solve_triu(U,y)
print(x)

[ 1.91812865  1.96491228 -0.98830409 -3.19298246 -1.13450292]


### Fatoração LU por redução de Doolittle


Uma forma de obter a fatoração $LU$ da matriz $A$, conhecido como processo Redução de Doolittle será apresentado a seguir ([fonte](https://integrada.minhabiblioteca.com.br/reader/books/9788522112821/pageid/43)). Considere a fatoração da matriz $A = (a_{ij})_{i,j=1,...,n}$ nas matrizes $L = (l_{ij})_{i,j=1,...,n}$ e $U = (u_{ij})_{i,j=1,...,n}$ como é mostrado a seguir

$$\left[\begin{array}{cccc} 
	         1      & 0      & \cdots & 0      \\ 
	         l_{21} & 1      & \cdots & 0      \\
	         \vdots & \vdots & \vdots & \vdots \\
	         l_{n1} & l_{n2} & \cdots & 1 \\
	         \end{array} \right] 
\left[\begin{array}{cccc} 
	         u_{11} & u_{12} & \cdots & u_{1n} \\ 
	         0      & u_{22} & \cdots & u_{2n} \\
	         \vdots & \vdots & \vdots & \vdots \\
	         0      & 0      & \cdots & u_{nn} \\
	         \end{array} \right]
	         =
\left[\begin{array}{cccc} 
	         a_{11} & a_{12} & \cdots & a_{1n} \\ 
	         a_{21} & a_{22} & \cdots & a_{2n} \\
	         \vdots & \vdots & \vdots & \vdots \\
	         a_{n1} & a_{n2} & \cdots & a_{nn} \\
	         \end{array} \right]$$

o processo de múltiplicação de matrizes nos leva às seguintes equações, que fornecem a primeira linha da matriz $U$: $u_{11}=a_{11}$, $u_{12}=a_{12}$, $u_{12}=a_{12}$,..., $u_{1n}=a_{1n}$. Seguindo o procedimento de multiplicar matrizes, podemos encontrar a 1ª coluna da matriz $L$: $l_{21}=a_{21}/u_{11}$, $l_{31}=a_{31}/u_{11}$,..., $u_{n1}=a_{n1}/u_{11}$. Continuando o processo para a 2ª linha de $U$, 2ª coluna de $L$, 3ª linha de $U$, 3ª coluna de $L$, e assim por diante, chegamos nas seguintes fórmulas:
$$u_{ij}=a_{ij}-\sum\limits_{k=1}^{i-1}l_{ik}u_{kj} \qquad i,j=1,...,n$$
e
$$l_{ij}=\frac{a_{ij}-\sum\limits_{k=1}^{j-1}l_{ik}u_{kj}}{ujj} \qquad i,j=1,...,n$$

Uma implementação desse algoritmo é mostrado a seguir.

In [40]:
def fatoracao_LU(A):
    n = len(A)     
    U = np.zeros((n,n))
    L = np.identity(n)
    for m in range(n):
        for j in range(m, n):
            U[m,j] = A[m,j] - np.sum(L[m,0:m] * U[0:m,j])
        for i in range(m+1, n):
            L[i,m] = (A[i,m] - np.sum(L[i,0:m] * U[0:m,m]))/U[m,m]
    return L,U

Testando para a mesma matriz do exemplo 3.

In [41]:
A = np.array([[2,0,1],
              [0,2,1],
              [1,1,3]])
L,U = fatoracao_LU(A)

print ("Matriz L:")
print (np.array(L))
print ("Matriz U:")
print(np.array(U))

Matriz L:
[[1.  0.  0. ]
 [0.  1.  0. ]
 [0.5 0.5 1. ]]
Matriz U:
[[2. 0. 1.]
 [0. 2. 1.]
 [0. 0. 2.]]


**Exercícios ([fonte](https://integrada.minhabiblioteca.com.br/reader/books/9788522123414/pageid/464)):**



**1.** Determine a fatoração $LU$ da matriz $A$ e resolva o sistema linear $Ax = b$, em que 

$$
A=\left[\begin{array}{rrrr}
1 & 1 & 0 & 3 \\
2 & 1 & -1 & 1 \\
3 & -1 & -1 & 2 \\
-1 & 2 & 3 & -1
\end{array}\right] \quad \text { e } \quad \mathbf{b}=\left[\begin{array}{r}
4 \\
1 \\
-3 \\
4
\end{array}\right]
$$


**2.** Use a fatoração do exercício anterior para resolver o sistema 
$$
\begin{aligned}
x_1+x_2+3 x_4 & =8, \\
2 x_1+x_2-x_3+x_4 & =7, \\
3 x_1-x_2-x_3+2 x_4 & =14, \\
-x_1+2 x_2+3 x_3-x_4 & =-7 .
\end{aligned}
$$

### Usando o módulo `scipy.linalg` para fatorar uma matriz
https://docs.scipy.org/doc/scipy/reference/linalg.html

In [44]:
import scipy
import scipy.linalg

In [45]:
A = np.array([[2,0,1],
              [0,2,1],
              [1,1,3]])

In [46]:
scipy.linalg.lu(A)

(array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 array([[1. , 0. , 0. ],
        [0. , 1. , 0. ],
        [0.5, 0.5, 1. ]]),
 array([[2., 0., 1.],
        [0., 2., 1.],
        [0., 0., 2.]]))

In [53]:
A = np.array([[1,2,4],[2,8,10],[4,10,26]])
b = np.array([1,-4,10])

In [54]:
R = np.linalg.cholesky(A)
R

array([[1., 0., 0.],
       [2., 2., 0.],
       [4., 1., 3.]])

In [55]:
R@R.T

array([[ 1.,  2.,  4.],
       [ 2.,  8., 10.],
       [ 4., 10., 26.]])

In [60]:
Q,R = np.linalg.qr(A)
print(Q)
print(R)

[[-0.21821789  0.27263927 -0.93704257]
 [-0.43643578 -0.88607763 -0.15617376]
 [-0.87287156  0.374879    0.31234752]]
[[ -4.58257569 -12.65663763 -27.93188995]
 [  0.          -2.79455252   1.97663471]
 [  0.           0.           2.81112771]]


In [61]:
Q@R

array([[ 1.,  2.,  4.],
       [ 2.,  8., 10.],
       [ 4., 10., 26.]])

### Exercício

**1.** Obtenha, caso exista, a solução para os seguintes sistemas lineares utilizando os métodos estudados (eliminação de Gauss e decomposição LU).


a) $\begin{cases} 
	         2x_1 + 3x_2 + x_3 +5x_4= 11\\ 
	         x_1  + 3.5x_2  + x_3 +7.5x_4= 13\\
	         1.4x_1 + 2.7x_2 + 5.5x_3 + 12x_4 = 21.6\\
	         -2x_1 + 1x_2 + 3x_3 +28x_4 = 30
	         \end{cases}$
    
b) $\begin{cases} 
	         6.1x_1 + 0.32x_2 + 1.3x_3 +2.1x_4 + 0.11x_5 = 19.52\\ 
	         0.82x_1  + 8.81x_2  + 1.01x_3 +3x_4 + 3.12x_5= 15.83\\
	         0.5x_1 + 1.78x_2 + 15.2x_3 + 4.2x_4 +8.1x_5= -22.14\\
	         4.2x_1 + 5.3x_2 + 1.8x_3 +20.9x_4 +7.51x_5 = 27.28\\
	         0.2x_1 + 9.1x_2 + 4.68x_3 +4.3x_4 +20.1x_5  = -21.78
	         \end{cases}$
             
c)$ \begin{cases} 
    12.1756 x_1 + 4.0231 x_2 - 2.1732 x_3 + 5.1967 x_4 = 17.1020\\ 
   -4.0231 x_1 + 6.0030 x_2              + 1.1973 x_4 = -6.1593\\
   -1.0000 x_1 - 5.2107 x_2 + 11.1111 x_3              =  3.0004\\
	6.0235 x_1 + 7.0000 x_2 +            - 14.1561 x_4 =  0.0000
\end{cases} $

**2.** ([Burden et al, 2016, p. 420](https://integrada.minhabiblioteca.com.br/reader/books/9788522123414/pageid/436)) Resolva o seguinte sistema linear por eliminação de Gauss e decomposição LU (se possível).

$$
\begin{aligned}
1,19 x_1+2,11 x_2-100 x_3+x_4 & =1,12 \\
14,2 x_1-0,122 x_2+12,2 x_3-x_4 & =3,44 \\
100 x_2-99,9 x_3+x_4 & =2,15 \\
15,3 x_1+0,110 x_2-13,1 x_3-x_4 & =4,16
\end{aligned}
$$


**3.** ([Burden et al, 2016, p.455](https://integrada.minhabiblioteca.com.br/reader/books/9788522123414/pageid/471)) Use fatoração LU para resolver o sistema linear a seguir. 

$$
\begin{aligned}
2 x_1+x_2 & =0, \\
-x_1+3 x_2+3 x_3 & =5, \\
2 x_1-2 x_2+x_3+4 x_4 & =-2, \\
-2 x_1+2 x_2+2 x_3+5 x_4 & =6 .
\end{aligned}
$$